# Visualizing Distribution of Bus Stops in Vancouver

### Started and Completed on May 13th, 2020

I use two separate APIs in this project, one from Translink, which iterates over 400 coordinate pairs in metro vancouver and pulls in a list of all the bus stops. I then use a second API from mapquest to get a map of metro 
vancouver, which I can then plot all of my coordi

In [ ]:
from TranslinkAccess import TranslinkAPI

TAPI = TranslinkAPI() 